## 1. RAG 기본
Let’s look at adding in a retrieval step to a prompt and LLM, which adds up to a “retrieval-augmented generation” chain

In [ ]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores import FAISS
import os
import pprint

단일 질문/응답 테스트

In [ ]:
'''
api_key = "<본인 api key 사용>"
os.environ["OPENAI_API_KEY"] = api_key
'''

In [ ]:
# model, embedding, vector store, retriever 설정
model = ChatOpenAI()
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(["harrison worked at kensho"], embedding=embedding)
retriever = vectorstore.as_retriever()

In [ ]:
# prompt 지정
template = """
Answer the question based only on the following context:
{context}
Question: 
{question}
"""
prompt = ChatPromptTemplate.from_template(template)
# print(prompt.input_variables)
# print(prompt.messages[0].prompt)

In [ ]:
# chain 지정
chain = (
    {"context": retriever, 
     "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# 실행
chain.invoke("where did harrison work?")

In [ ]:
template = """
Answer the question based only on the following context:
{context}
Question: {question}
Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question"),
     "language": itemgetter("language")}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke({"question": "where did harrison work", "language": "korean"})

## 2. Conversational Retrieval Chain

In [ ]:
from langchain.schema import format_document
from langchain.schema.messages import get_buffer_string
from langchain.schema.runnable import RunnableParallel
from langchain_core.messages import AIMessage, HumanMessage
from langchain.prompts.prompt import PromptTemplate

In [ ]:
_template = """
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [ ]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [ ]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [ ]:
conversational_qa_chain.invoke(
    {
        "question": "where did harrison work?",
        "chat_history": [],
    }
)

In [ ]:
AIMessage(content='Harrison was employed at Kensho.')

In [ ]:
conversational_qa_chain.invoke(
    {
        "question": "where did he work?",
        "chat_history": [
            HumanMessage(content="Who wrote this notebook?"),
            AIMessage(content="Harrison"),
        ],
    }
)

In [ ]:
AIMessage(content='Harrison worked at Kensho.')

In [ ]:
AIMessage.__doc__